This is my implementation of Heeger Bergen texture synthesis algorithm with downsampling and upsampling. It is based on paper given http://www.ipol.im/pub/art/2014/79/article_lr.pdf.

# Load Libraries

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.image as imgplt
import matplotlib.image as mpimg

# Make Filters

In [ ]:
def L(r):
    if (r <= np.pi/4):
        return 1
    elif (r <= np.pi/2):
        return np.cos(np.pi/2 * np.log2(4*r/np.pi))
    else:
        return 0

def H(r):
    if (r <= np.pi/4):
        return 0
    elif (r <= np.pi/2):
        return np.cos(np.pi/2 * np.log2(2*r/np.pi))
    else:
        return 1

def alpha(Q):
    return 2**(Q-1) * math.factorial(Q-1) / np.sqrt(Q * math.factorial(2*Q-2))

def G(q,Q,theta): 
    r1 = theta - np.pi * q/Q
    if (r1 < -np.pi):
        r1 = r1 + 2*np.pi
    region1 = np.abs(r1)
    r2 = theta - np.pi * (q-Q)/Q
    if (r2 > np.pi):
        r2 = r2 - 2*np.pi
    region2 = np.abs(r2)
    if (region1 <= np.pi/2 and region2 <= np.pi/2):
        return alpha(Q) * np.cos(r1)**(Q-1) + np.cos(r2)**(Q-1)
    elif (region1 <= np.pi/2 and region2 >= np.pi/2):
        return alpha(Q) * np.cos(r1)**(Q-1)
    elif (region1 >= np.pi/2 and region2 <= np.pi/2):
        return alpha(Q) * np.cos(r2)**(Q-1)
    else:
        return 0

def B(q,Q,r,theta):
    return H(r)*G(q,Q,theta)

def calc_r(x,y, theta = 'yes'):
    if(theta == 'no'):
        if(y == 0 and x <= 0):
            r = np.abs(x)
        else:
            r = np.sqrt(x**2 + y**2)
        return r
    else:
        if(y == 0 and x <= 0):
            r = np.abs(x)
            theta = np.pi
        else:
            r = np.sqrt(x**2 + y**2)
            theta = 2*np.arctan(y/(x+r))
        return r, theta

In [ ]:
def gen_H0mat(M,N):
    H0mat = np.zeros((M,N), dtype = 'float')
    for m in range(-int(M/2), int(M/2)):
        for n in range(-int(N/2), int(N/2)):
            x = 2*m*np.pi/M
            y = 2*n*np.pi/N
            r = calc_r(x, y, theta = 'no')
            H0mat[m,n] = H(r/2)
    H0mat = np.fft.fftshift(H0mat)
    return H0mat

def gen_L0mat(M,N):
    L0mat = np.zeros((M,N), dtype = 'float')
    for m in range(-int(M/2), int(M/2)):
        for n in range(-int(N/2), int(N/2)):
            x = 2*m*np.pi/M
            y = 2*n*np.pi/N
            r = calc_r(x, y, theta = 'no')
            L0mat[m,n] = L(r/2)
    L0mat = np.fft.fftshift(L0mat)
    return L0mat

def gen_Lmat(M,N,P):
    Lmat = []
    for p in range(0,P):
        matrix = np.zeros((M, N), dtype = 'float')
        for m in range(-int(M/2), int(M/2)):
            for n in range(-int(N/2), int(N/2)):
                x = 2*m*np.pi/M
                y = 2*n*np.pi/N
                r = calc_r(x, y, theta = 'no')
                matrix[m,n] = L(r)
        #print(matrix) 
        Lmat.append(np.fft.fftshift(matrix))
        M = int(M/2)
        N = int(N/2)
    return Lmat

def gen_Hmat(M,N,P):
    Hmat = []
    for p in range(0,P):
        matrix = np.zeros((M, N), dtype = 'float')
        for m in range(-int(M/2), int(M/2)):
            for n in range(-int(N/2), int(N/2)):
                x = 2*m*np.pi/M
                y = 2*n*np.pi/N
                r = calc_r(x, y, theta = 'no')
                matrix[m,n] = H(r)
        #print(matrix) 
        Hmat.append(np.fft.fftshift(matrix))
        M = int(M/2)
        N = int(N/2)
    return Hmat


def gen_Bmat(M,N,P,Q):
    Bmat = []
    for p in range(P):
        for q in range(Q):
            matrix = np.zeros((M, N), dtype = 'float')
            for m in range(-int(M/2), int(M/2)):
                for n in range(-int(N/2), int(N/2)):
                    x = 2*m*np.pi/M
                    y = 2*n*np.pi/N
                    r, theta = calc_r(x, y, theta = 'yes')
                    matrix[m,n] = B(q,Q,r,theta)
            Bmat.append(np.fft.fftshift(matrix))
        M = int(M/2)
        N = int(N/2)
    return Bmat

# Test Filters

In [ ]:
P = 4
Q = 4
H_set = gen_Hmat(256, 256, P)
L_set = gen_Lmat(256, 256, P)
plt.figure(figsize = (2*P,2*P))
c = 1
for j in range(P):
    plt.subplot(2,2,c)
    plt.imshow(H_set[j], extent = [-np.pi, np.pi, -np.pi, np.pi])
    plt.colorbar()
    c = c+1

In [ ]:
c = 1
plt.figure(figsize = (2*P,2*P))
for j in range(P):
    plt.subplot(2,2,c)
    plt.imshow(L_set[j], extent = [-np.pi, np.pi, -np.pi, np.pi])
    plt.colorbar()
    c = c+1

Now, it's time to check if the filter actually works. We'll check the condition that $$H(r)^2 +L(r)^2 = 1.$$

In [ ]:
P = 4
Q = 4
plt.figure(figsize = (2*P,2*P))
c = 1
for j in range(P):
    plt.subplot(2,2,c)
    plt.imshow(np.abs(H_set[j])**2 + np.abs(L_set[j])**2, extent = [-np.pi, np.pi, -np.pi, np.pi])
    plt.colorbar()
    c = c+1

Now let's check if $L_0(r)^2 + H_0(r)^2 = 1$

In [ ]:
H0 = gen_H0mat(256, 256)
L0 = gen_L0mat(256, 256)
plt.figure(figsize = (15,4))
plt.subplot(1,3,1)
plt.imshow(H0, extent = [-np.pi, np.pi, -np.pi, np.pi])
plt.colorbar()
plt.subplot(1,3,2)
plt.imshow(L0, extent = [-np.pi, np.pi, -np.pi, np.pi])
plt.colorbar()
plt.subplot(1,3,3)
plt.imshow(H0**2 + L0**2, extent = [-np.pi, np.pi, -np.pi, np.pi])
plt.colorbar();

# Plot

In [ ]:
P = 4
Q = 4
filters = gen_Bmat(256,256,P,Q)

plt.figure(figsize = (4*P,4*P))
c = 1
for j in range(P):
    for q in range(Q):
        plt.subplot(P, Q, c)
        plt.imshow(np.real(filters[4*j + q]), extent = [-np.pi, np.pi, -np.pi, np.pi])
        plt.colorbar()
        plt.title('j = ' + str(j) + ' q = ' + str(q))
        c = c + 1

# Downsample

In [ ]:
def downsample(img):
    M = img.shape[0]
    N = img.shape[1]
    img_downsample = np.zeros((int(M/2), int(N/2)), dtype = 'float')
    for m in range(int(M/2)):
        for n in range(int(N/2)):
            img_downsample[m,n] = img[2*m, 2*n]
    return img_downsample

def zero_pad(img):
    M = img.shape[0]
    N = img.shape[1]
    new_img = np.pad(img, ((int(M/2),int(M/2)),(int(N/2),int(N/2))), 'constant')
    return new_img

# Load Image

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# img_orig = imgplt.imread('/content/drive/My Drive/download.jpg')
# print(img_orig.shape)
# img_orig = mpimg.imread('butterfly.jpg')
img_orig = mpimg.imread('D4.gif')
img = img_orig[:512, :512] / 255
plt.imshow(img, cmap = 'gray')
plt.colorbar()

# Constructing Pyramid Algorithm

In [ ]:
def steerable_pyramid(img, num_scales, num_orientations, H0, L0, L_matrix, H_matrix, B_matrix):
    
    # Initialize
    count = 0
    pyramid = [] #empty list, will append
    
    # Compute the FFT of the image
    img_fft = np.fft.fft2(img)
    
    # High frequency residual coefficients
    u = np.fft.fftshift(H0) * img_fft
    u = np.fft.ifft2(u)
    u = np.real(u)
    pyramid.append(u)
    
    # Low frequency coefficients
    v = np.fft.fftshift(L0) * img_fft
    
    # Bandpass coefficients
    for p in range(num_scales):
        for q in range(num_orientations):
            out = np.fft.fftshift(B_matrix[count]) * v
            out = np.fft.ifft2(out)
            out = np.real(out)
            pyramid.append(out) 
            count = count + 1
        
        v = L_matrix[p] * np.fft.fftshift(v)
        M,N = v.shape
        v = v[int(M/4):int(3*M/4), int(N/4):int(3*N/4)]
        v = np.fft.fftshift(v) / 4
        
#         v = np.fft.fftshift(L_matrix[p]) * v
#         v = np.fft.ifft2(v)
#         v = np.real(v)
#         v = downsample(v)
#         if p < (num_scales-1):
#             v = np.fft.fft2(v)
    
    pyramid.append(np.fft.ifft2(v))
    return pyramid

In [ ]:
P = 4
Q = 4
M,N = img.shape
L0mat = gen_L0mat(M,N)
H0mat = gen_H0mat(M,N)
Lmat = gen_Lmat(M,N,P)
Hmat = gen_Hmat(M,N,P)
Bmat = gen_Bmat(M,N,P,Q)
pyra = steerable_pyramid(img, P, Q, H0mat, L0mat, Lmat, Hmat, Bmat)

# Example Pyramid

In [ ]:
plt.figure()
plt.figure(figsize = (4*P, 4*Q))
count = 1
for j in range(P):
    for q in range(Q):
        plt.subplot(P, Q, count)
        plt.imshow(np.real(pyra[count]), cmap = 'gray')
        plt.colorbar()
        plt.title('j = ' + str(j) + ' q = ' + str(q))
        count = count + 1

In [ ]:
plt.figure()
plt.figure(figsize = (12, 8))
plt.subplot(2, 2, 1)
plt.imshow(np.real(pyra[0]), cmap = 'gray')
plt.colorbar()
plt.subplot(2, 2, 2)
plt.imshow(np.real(pyra[P*Q+1]), cmap = 'gray')
plt.colorbar()


# Pyramid Inverse Algorithm

In [ ]:
def pyramid_reconstruct(pyramid, num_scales, num_orientations, L0, H0, Lmat, Hmat, Bmat):
    
    count = num_scales * num_orientations + 1
    u = pyramid[count]
    count = count-1
    for p in range(num_scales,0,-1):
        # u = upsample(u)
        uk = np.fft.fftshift(np.fft.fft2(u))
        uk = zero_pad(uk)
        uk = uk * Lmat[p-1]
        u = 4 * np.fft.ifft2(np.fft.fftshift(uk))
#         u = np.real(u)
        for q in range(num_orientations,0,-1):
            conv = np.fft.fftshift(Bmat[count-1]) * np.fft.fft2(pyramid[count])
            conv = np.fft.ifft2(conv)
#             conv = np.real(conv)
            u += conv
            count = count-1
    u = np.fft.fft2(u) * np.fft.fftshift(L0)
    u = np.fft.ifft2(u)
#     u = np.real(u)
    high = np.fft.fftshift(H0) * np.fft.fft2(pyramid[count])
    high = np.fft.ifft2(high)
#     high = np.real(high)
    u += high
    u = np.real(u)
    return u

In [ ]:
out = pyramid_reconstruct(pyra, P, Q, L0mat, H0mat, Lmat, Hmat, Bmat)
plt.imshow(out, cmap = 'gray')
plt.colorbar()
for i in range(P,0,-1):
    print(i)
print(np.mean(out))

# Histogram Matching

In [ ]:
def hist_match(u, v): #u and v are images of the same size
    v_ravel = v.ravel()
    u_ravel = u.ravel()

    v_sorted = np.sort(v_ravel) # sort in ascending order
    # u_sorted = np.sort(u_ravel)

    tau = np.argsort(v_ravel)
    sigma = np.argsort(u_ravel)

    u_new_vec = np.zeros(v_ravel.shape[0]) #new vector after matching
    for i in range(v_ravel.shape[0]):
#         u_new_vec[tau[i]] = v_ravel[sigma[i]] #replace like in paper
        u_new_vec[sigma[i]] = v_ravel[tau[i]] #replace like in paper

    u_new = np.reshape(u_new_vec, (u.shape[0], u.shape[1]), order = 'C') 
    return u_new

# Make White Noise

In [ ]:
def gen_white_noise(img):
    noise = np.zeros((img.shape[0], img.shape[1]), dtype = 'float')
    for n in range(img.shape[0]):
        for m in range(img.shape[1]):
            noise[n][m] = np.random.normal(0, 0.5)
    return noise

# Full Algorithm

In [ ]:
def heeger_bergen(img, P, Q, iterations, L0, H0, Lmat, Hmat, Bmat):
    
    # Get the dimensions of the image
    M,N = np.shape(img)
    
    # Generate the filters
#     H_mat = H_mat_struct(M,N,P)
#     L_mat = L_mat_struct(M,N,P+1)
#     B_mat = B_mat_struct(M,N,P+1,Q)

    # Store each synthesis iteration
    v = np.zeros((M,N,iterations+1))
    
    # Synthesis algorithm
    img_pyra = steerable_pyramid(img, P, Q, H0, L0, Lmat, Hmat, Bmat)
    v[:,:,0] = gen_white_noise(img)
    v[:,:,0] = hist_match(v[:,:,0], img)
    for n in range(iterations):
        noise_pyra = steerable_pyramid(v[:,:,n], P, Q, H0, L0, Lmat, Hmat, Bmat)
        for j in range(P*Q+2):
            noise_pyra[j] = hist_match(np.real(noise_pyra[j]), np.real(img_pyra[j]))
        v[:,:,n+1] = pyramid_reconstruct(noise_pyra, P, Q, L0, H0, Lmat, Hmat, Bmat)
        v[:,:,n+1] = hist_match(v[:,:,n+1], img)  
    return v

In [ ]:
num_iter = 5
out = heeger_bergen(img, P, Q, num_iter, L0mat, H0mat, Lmat, Hmat, Bmat)
plt.imshow(np.real(out[:,:,5]), cmap = 'gray')
plt.colorbar()

In [ ]:
# Display and compare

for n in range(num_iter + 1):
    
    plt.figure(figsize = (18,5))
    
    plt.subplot(1,3,1)
    plt.imshow(img, cmap = 'gray')
    plt.colorbar()
    plt.title('Original image')

    plt.subplot(1,3,2)
    plt.imshow(out[:,:,n], cmap = 'gray')
    plt.colorbar()
    plt.title('Synthesized texture')

    plt.subplot(1,3,3)
    plt.imshow(np.abs(img - out[:,:,n]), cmap = 'gray')
    plt.colorbar()
    plt.title('Difference of images')